Part 1 70%
Write a jupyter notebook that analyses the differences between the sexes by age in Ireland.

Weighted mean age (by sex)
The difference between the sexes by age
This part does not need to look at the regions.

ie You can take the notebook I used in the lectures and substitute the sexes for the regions.

Part 2 20%
In the same notebook, make a variable that stores an age (say 35).

Write that code that would group the people within 5 years of that age together, into one age group 

Calculate the population difference between the sexes in that age group.

Part 3 10%
In the same notebook.

Write the code that would work out which region in Ireland has the biggest population difference between the sexes in that age group

In [179]:
import pandas as pd

In [180]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)
df.head()

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
0,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,IE0,Ireland,Number,5149139
1,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-1492-13a3-e055-000000000001,Carlow County Council,Number,61968
2,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-1433-13a3-e055-000000000001,Dublin City Council,Number,592713
3,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-149f-13a3-e055-000000000001,Dún Laoghaire Rathdown County Council,Number,233860
4,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-14a0-13a3-e055-000000000001,Fingal County Council,Number,330506


In [181]:
headers = df.columns.tolist()
headers

['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'Sex',
 'C02076V03371',
 'Single Year of Age',
 'C03789V04537',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [182]:
drop_col_list = ['STATISTIC', 'Statistic Label','TLIST(A1)','CensusYear','C02199V02655','C02076V03371','C03789V04537','UNIT']
df.drop(columns=drop_col_list, inplace=True)
df = df[df["Single Year of Age"] != "All ages"]
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')
df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)

df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df['VALUE'] = df['VALUE'].round(0).astype('int64')

# Round all numeric columns to nearest integer and convert to int
df[df.select_dtypes(include='number').columns] = df.select_dtypes(include='number').round(0).astype('int64')
#print (df.head(3))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9696 entries, 32 to 9791
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      9696 non-null   object
 1   Single Year of Age       9696 non-null   int64 
 2   Administrative Counties  9696 non-null   object
 3   VALUE                    9696 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 378.8+ KB


<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
C:\Users\finia\AppData\Local\Temp\ipykernel_14524\2744731524.py:5: SyntaxWarning: invalid escape sequence '\D'
  df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)


In [183]:
df_anal = pd.pivot_table(df, 'VALUE',"Single Year of Age","Sex")

# Ensure numeric values are integers
df_anal[df_anal.select_dtypes(include='number').columns] = df_anal.select_dtypes(include='number').round(0).astype('int64')


print (df_anal.head(5))
# write out the entire file to local machine
df_anal.to_csv("population_for_analysis.csv")

Sex                 Both sexes  Female  Male
Single Year of Age                          
0                         3612    1762  1851
1                         3526    1722  1805
2                         3701    1811  1890
3                         3780    1843  1938
4                         3844    1864  1980


In [184]:
headers = list(df_anal.columns)
Both_sexes = headers[0]
female = headers[1]
male = headers[2]

print("Columns in pivoted dataframe:", headers)

Columns in pivoted dataframe: ['Both sexes', 'Female', 'Male']


In [185]:
number_males = df_anal['Male'].sum()
number_females = df_anal['Female'].sum()
number_both = df_anal['Both sexes'].sum()   
print("Total males:", number_males)
print("Total females:", number_females)
print("Total both sexes:", number_both)
district = headers[0]

Total males: 159043
Total females: 162790
Total both sexes: 321823


In [186]:
mean_males = df_anal['Male'].mean()
mean_females = df_anal['Female'].mean()
mean_both = df_anal[Both_sexes].mean()

# Convert to integers (rounded) for display
mean_males_int = int(round(mean_males))
mean_females_int = int(round(mean_females))
mean_both_int = int(round(mean_both))

print("mean-males:", mean_males_int)
print("mean-females:", mean_females_int)
print("mean-both:", mean_both_int)

mean-males: 1575
mean-females: 1612
mean-both: 3186
